## knock 068 乗算結果に対して有効桁数未満を切り捨てる

In [7]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [8]:
df_product = pl.read_csv("../docker/work/data/product.csv")
display(df_product.head())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
str,i64,i64,i64,i64,i64
"""P040101001""",4,401,40101,198,149
"""P040101002""",4,401,40101,218,164
"""P040101003""",4,401,40101,230,173
"""P040101004""",4,401,40101,248,186
"""P040101005""",4,401,40101,268,201


### ノック

In [9]:
(
    df_product
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select([
        pl.col("product_cd"),
        pl.col("unit_price")
    ])
    # 税込み金額を端数切り上げで計算する
    .with_columns(
        (pl.col("unit_price") * 1.1)
        .floor()
        .cast(pl.Int64)
        .alias("tax_price")
    )
    # 先頭10行
    .head(n = 10)
)

product_cd,unit_price,tax_price
str,i64,i64
"""P040101001""",198,217
"""P040101002""",218,239
"""P040101003""",230,253
"""P040101004""",248,272
"""P040101005""",268,294
"""P040101006""",298,327
"""P040101007""",338,371
"""P040101008""",420,462
"""P040101009""",498,547
